In [ ]:
import requests, json, sys, math, time
import pandas as pd
from typing import List

In [ ]:
baseUrl = 'https://www.tonal.com'

In [ ]:
def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   
   return "%s %s" % (s, size_name[i])

## REST

In [ ]:
rest_list = []

In [ ]:
for i in range(1, 26):
    start = time.time()

    posts_response = requests.get(f'{baseUrl}/wp-json/wp/v2/posts',params={'per_page':i})

    response_data: List = json.loads(posts_response.text)
    author_ids = [str(x['author']) for x in response_data]

    authors_response = requests.get(f'{baseUrl}/wp-json/wp/v2/users',params={'include':','.join(author_ids)})

    response_size = sys.getsizeof(posts_response.text + authors_response.text)
    formatted_response_size = convert_size(response_size)

    execution_time = round(time.time() - start, 2)

    rest_list.append({'per_page': i, 'response_size': formatted_response_size, 'execution_time': execution_time})

In [ ]:
a = pd.DataFrame(rest_list)
a

## GraphQL

In [ ]:
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport

In [ ]:
a = Client(transport=gql.transport.RequestsTransport())

for i in range(1,11):
    start = time.time()

    
    query=r"""
    query {
        posts(first: {first}) {
            nodes {
                id
                title
                author {
                    node {
                        name
                    }
                }
            }
        }
    }
    """.format(first=i)

    print('query: ', query)

    # response = requests.post(f'{baseUrl}/graphql',headers={'Content-Type': 'application/json'},data=json.dumps({'query': query}))

    # response_data: List = json.loads(posts_response.text)
    # author_ids = [str(x['author']) for x in response_data]

    # authors_response = requests.get(f'{baseUrl}/wp-json/wp/v2/users',params={'include':','.join(author_ids)})

    # print('authors_response: ', authors_response)

    # data = sys.getsizeof(posts_response.text + authors_response.text)
    # a = convert_size(data)

    # execution_time = round(time.time() - start, 2)
    # print(f'per page: {i}, size: {a}. time: {execution_time} seconds')